In [1]:
from param import ml_names, pers_names, data_name, data_path, results_path, thres_sel_feat
from ml_models import loocv_day, agg_feat_imp
from ml_models import MAE, MSE, RMSE, MAPE, R2
import pandas as pd
import numpy as np
import os

In [2]:
os.chdir(results_path)
df_ml_input_new = pd.read_csv('ml_input_day.csv', index_col = 0)

In [ ]:
ml_model_name = 'xgb'
ls_results_xgb = list()
ls_feats_xgb = list()
for pers_name in pers_names:
    ls_results_loocv_xgb, ls_feats_loocv_xgb = loocv_day(df_ml_input_new, pers_name, ml_model_name)
    ls_results_xgb.extend(ls_results_loocv_xgb)
    ls_feats_xgb.extend(ls_feats_loocv_xgb)
    df_results_xgb_raw = pd.DataFrame(ls_results_loocv_xgb, columns = ['Personality', 'Participant ID', 'Y test', 'Y pred', 'Best param'])
    df_feats_xgb_raw = pd.DataFrame()
    df_feats_xgb_raw['Personality'] = df_results_xgb_raw['Personality'].tolist()
    df_feats_xgb_raw['Participant ID'] = df_results_xgb_raw['Participant ID'].tolist()
    df_feats_xgb_raw['Feats'] = ls_feats_loocv_xgb
    df_results_xgb_raw.to_csv('xgb_' + pers_name + '_results_raw.csv', index=None)
    df_feats_xgb_raw.to_csv('xgb_' + pers_name + '_feats_raw.csv', index=None)

In [ ]:
df_results_xgb = pd.DataFrame(ls_results_xgb, columns = ['Personality', 'Participant ID', 'Y test', 'Y pred', 'Best param'])
df_results_xgb.head()

In [ ]:
ls_evalutation_xgb = list()
df_results_xgb_groups = df_results_xgb.groupby('Personality')
df_results_xgb_group_names = df_results_xgb_groups.size().index
for df_results_xgb_group_name in df_results_xgb_group_names:
    ls_evaluation_row_xgb = [df_results_xgb_group_name]
    df_results_xgb_group = df_results_xgb_groups.get_group(df_results_xgb_group_name)
    ls_test_vals_xgb = list()
    ls_pred_vals_xgb = list()
    for val in df_results_xgb_group['Y test'].tolist():
        if len(val) != 1:
            print('Error')
        
        ls_test_vals_xgb.append(val[0])

    for val in df_results_xgb_group['Y pred'].tolist():
        if len(val) != 1:
            print('Error')
            
        ls_pred_vals_xgb.append(val[0])
    
    df_evaluation_input_xgb = pd.DataFrame()
    df_evaluation_input_xgb['Y_test'] = ls_test_vals_xgb
    df_evaluation_input_xgb['Y_pred'] = ls_pred_vals_xgb
    
#     df_evalution_output_xgb = evaluate(df_evaluation_input_xgb)
    ls_evalutation_xgb.append([df_results_xgb_group_name, 
                               MAE(ls_pred_vals_xgb, ls_test_vals_xgb),
                               MSE(ls_pred_vals_xgb, ls_test_vals_xgb),
                               RMSE(ls_pred_vals_xgb, ls_test_vals_xgb),
                               MAPE(ls_pred_vals_xgb, ls_test_vals_xgb),
                               R2(ls_pred_vals_xgb, ls_test_vals_xgb)])

In [ ]:
df_results_summary = pd.DataFrame(ls_evalutation_xgb, columns = ['Personality', 'MAE', 'MSE', 'RMSE', 'MAPE(%)', 'R2'])
df_results_summary.to_csv('xgb_results_summary.csv', index=None)

In [ ]:
dict_feats_summary_imp_xgb, dict_feats_summary_freq_xgb = agg_feat_imp(ls_feats_xgb)

In [ ]:
ls_feats_summary_imp_xgb = list()
for feat_name in dict_feats_summary_imp_xgb:
    ls_row = [feat_name, dict_feats_summary_imp_xgb[feat_name]]
    ls_feats_summary_imp_xgb.append(ls_row)

df_feats_summary_imp_xgb = pd.DataFrame(ls_feats_summary_imp_xgb, columns = ['Feature', 'Importance'])
df_feats_summary_imp_xgb.to_csv('xgb_feats_imp_summary.csv',index=None)

In [ ]:
ls_feats_summary_freq_xgb = list()
for feat_name in dict_feats_summary_freq_xgb:
    ls_row = [feat_name, dict_feats_summary_freq_xgb[feat_name]]
    ls_feats_summary_freq_xgb.append(ls_row)

df_feats_summary_freq_xgb = pd.DataFrame(ls_feats_summary_freq_xgb, columns = ['Feature', 'Importance'])
df_feats_summary_freq_xgb.to_csv('xgb_feats_freq_summary.csv',index=None)